In [7]:
from collections import defaultdict
from pathlib import Path
import json
import os

from pymongo import MongoClient
import pandas as pd

In [2]:
data_dir = Path.cwd().parent / 'data' / 'output' / 'idn-tagged' / 'results'

In [3]:
overall_f1_scores = defaultdict(list)
for model_name in 'majority memo crf ctx-affix-crf'.split():
    for fold in range(1, 6):
        result_path = data_dir / f'result-{model_name}.{fold:02}.json'
        with open(result_path) as f:
            result = json.load(f)
        overall_f1_scores[model_name].append(result['overall_f1'])

In [8]:
# For neural tagger
client = MongoClient(os.environ['SACRED_MONGO_URL'])
db = client[os.environ['SACRED_DB_NAME']]

for fold in range(1, 6):
    run = db.runs.find_one({
        'experiment.name': 'id-pos-tagging-neural',
        'meta.command': 'evaluate',
        'status': 'COMPLETED',
        'config.corpus.test': {'$regex': f'test.{fold:02}.tsv$'},
        'config.eval.which': 'test',
    }, ['result'])
    overall_f1_scores['lstm+crf+pref+suff'].append(run['result'])

In [9]:
df = pd.DataFrame(overall_f1_scores, index=range(1, 6))

In [10]:
df

,crf,ctx-affix-crf,lstm+crf+pref+suff,majority,memo
1,0.960914,0.962078,0.974924,0.092034,0.920352
2,0.964923,0.959178,0.973200,0.092209,0.899710
3,0.962995,0.964004,0.976103,0.093065,0.904711
4,0.950597,0.964582,0.975228,0.095158,0.904421
5,0.962167,0.961157,0.973848,0.096895,0.901569


In [11]:
df.describe()

,crf,ctx-affix-crf,lstm+crf+pref+suff,majority,memo
count,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.960319,0.962200,0.974661,0.093872,0.906152
std,0.005627,0.002189,0.001147,0.002097,0.008204
min,0.950597,0.959178,0.973200,0.092034,0.899710
25%,0.960914,0.961157,0.973848,0.092209,0.901569
50%,0.962167,0.962078,0.974924,0.093065,0.904421
75%,0.962995,0.964004,0.975228,0.095158,0.904711
max,0.964923,0.964582,0.976103,0.096895,0.920352
